In [1]:
# install psycopg2
!pip install psycopg2-binary

     |████████████████████████████████| 2.2 MB 2.1 MB/s eta 0:00:01     |██▍                             | 163 kB 2.1 MB/s eta 0:00:01


In [2]:
# dependencies
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [3]:
csv_file = "../Resources/customer_data.csv"
customer_data_df = pd.read_csv(csv_file)
customer_data_df.head()

,id,first_name,last_name,email,gender,car
0,1,Benetta,Cancott,bcancott0@studiopress.com,Female,Scion
1,2,Lilyan,Cherry,lcherry1@deliciousdays.com,Female,Chrysler
2,3,Ezekiel,Benasik,ebenasik2@wikia.com,Male,Mercedes-Benz
3,4,Kennedy,Atlay,katlay3@so-net.ne.jp,Male,Buick
4,5,Sanford,Salmen,ssalmen4@reuters.com,Male,Lincoln


### Create new data with select columns

In [4]:
new_customer_data_df = customer_data_df[['id', 'first_name', 'last_name']].copy()
new_customer_data_df.head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Store JSON data into a DataFrame

In [5]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

,id,address,longitude,latitude,us_state
0,1,043 Mockingbird Place,-86.5186,39.1682,Indiana
1,2,4 Prentice Point,-85.0707,41.0938,Indiana
2,3,46 Derek Junction,-96.7776,32.7673,Texas
3,4,11966 Old Shore Place,-94.3567,39.0350,Missouri
4,5,5 Evergreen Circle,-73.9772,40.7808,New York


### Clean DataFrame

In [6]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


### Connect to local database

In [7]:
protocol = 'postgresql'
username = 'postgres'
password = 'admin'
host = 'localhost'
port = 5432
database_name = 'customer_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

### Check for tables

In [11]:
engine.table_names()

/Users/jasonnoble/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['customer_name', 'customer_location']

### Use pandas to load csv converted DataFrame into database

In [12]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='replace', index=False)

### Use pandas to load json converted DataFrame into database

In [13]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='replace', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [10]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [11]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


In [15]:
df = customer_location_df
df.head()

,id,address,longitude,latitude,us_state
0,1,043 Mockingbird Place,-86.5186,39.1682,Indiana
1,2,4 Prentice Point,-85.0707,41.0938,Indiana
2,3,46 Derek Junction,-96.7776,32.7673,Texas
3,4,11966 Old Shore Place,-94.3567,39.0350,Missouri
4,5,5 Evergreen Circle,-73.9772,40.7808,New York


In [16]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
id,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
address,043 Mockingbird Place,4 Prentice Point,46 Derek Junction,11966 Old Shore Place,5 Evergreen Circle,5304 Vidon Drive,65 Texas Avenue,041 Eagle Crest Court,72402 Annamark Terrace,7 Sauthoff Circle,...,6326 Erie Court,62 Lawn Park,39 Laurel Drive,1 Maple Wood Center,44968 Schiller Junction,6546 Waxwing Circle,604 Parkside Street,44 Northwestern Court,33 Bunting Point,8389 Sommers Street
longitude,-86.5186,-85.0707,-96.7776,-94.3567,-73.9772,-96.7676,-84.2024,-102.0915,-77.0089,-94.2382,...,-89.8826,-93.0828,-122.2698,-94.3944,-82.4388,-86.9292,-111.6627,-74.2452,-119.8225,-122.3748
latitude,39.1682,41.0938,32.7673,39.035,40.7808,32.7887,39.8285,32.0295,38.8992,35.191,...,30.033,44.9427,37.8044,38.8518,27.872,33.5446,40.7286,40.7918,39.5387,45.5806
us_state,Indiana,Indiana,Texas,Missouri,New York,Texas,Ohio,Texas,District of Columbia,Arkansas,...,Louisiana,Minnesota,California,Missouri,Florida,Alabama,Utah,New Jersey,Nevada,Oregon
